In [4]:
%load_ext autoreload
%autoreload 2

In [14]:
from itertools import product

import tqdm
import pandas as pd

from jaccard_recommender.recommender import JaccardRecommender
from IPython.display import clear_output

In [9]:
data_path = "../data/copperunion"
output_path = "../models/jaccard_knn"
results = {}
for normalize_unrated, thresholded_watch_history, true_weighted_average in tqdm.tqdm(
    product([True, False], [10, 20, 100], [True, False]),
    desc="grid search...",
    total=2 * 2 * 3,
):
    options = {
        "normalize_unrated": normalize_unrated,
        "thresholded_watch_history": thresholded_watch_history,
        "true_weighted_average": true_weighted_average,
    }
    print(f"{normalize_unrated=} {thresholded_watch_history=} {true_weighted_average=}")
    recommender = JaccardRecommender(data_path, output_path, **options)
    recommender.build_matrix()
    runtime, ndcg = recommender.inference()
    results[normalize_unrated, thresholded_watch_history, true_weighted_average] = (
        runtime,
        ndcg,
    )
clear_output()

grid search...:   0%|          | 0/12 [00:00<?, ?it/s]

normalize_unrated=True thresholded_watch_history=10 true_weighted_average=True
normalize_unrated=True


parsing users...: 100%|██████████| 73515/73515 [00:21<00:00, 3384.32it/s]


Total Animes: 12294
Total Users: 61674


building interaction matrix...: 100%|██████████| 49339/49339 [00:06<00:00, 8016.93it/s]


...

building interaction matrix...: 100%|██████████| 19757/19757 [00:03<00:00, 5220.71it/s]


Percentage Zeroes: 0.41
(2471, 10)


grid search...: 100%|██████████| 12/12 [14:06<00:00, 70.52s/it]

This model took 10.2462 seconds.
Out of an optimal score of 1.0, you scored 0.1622.


NameError: name 'clear_output' is not defined

In [16]:
data = [list(k) + list(v) for k, v in results.items()]
df = pd.DataFrame(
    data,
    columns=[
        "normalize_unrated",
        "thresholded_watch_history",
        "true_weighted_average",
        "runtime",
        "ndcg @ 10",
    ],
)
display(df)

,normalize_unrated,thresholded_watch_history,true_weighted_average,runtime,ndcg @ 10
0,True,10,True,37.234467,0.001336
1,True,10,False,24.208000,0.232263
2,True,20,True,34.178117,0.001621
3,True,20,False,24.858905,0.228416
4,True,100,True,18.773455,0.002093
5,True,100,False,11.286252,0.184946
6,False,10,True,45.141935,0.001396
7,False,10,False,29.634101,0.204085
8,False,20,True,33.149710,0.001523
9,False,20,False,24.029245,0.203184


In [ ]:
options = {
    "normalize_unrated": True,
    "thresholded_watch_history": 20,
    "true_weighted_average": True,
}
recommender = JaccardRecommender(data_path, output_path, **options)
recommender.build_matrix()
runtime, ndcg = recommender.inference()
print(runtime, ndcg)

normalize_unrated=False thresholded_watch_history=100 true_weighted_average=False


parsing users...: 100%|██████████| 73515/73515 [00:15<00:00, 4706.57it/s]


Total Animes: 12294
Total Users: 54077


building interaction matrix...: 100%|██████████| 43261/43261 [00:03<00:00, 11111.90it/s]


Percentage Zeroes: 0.41
10074
2220
12294
float64
